In [1]:
! pip install transformers
! pip install torch torchvision torchaudio

In [2]:
# MODEL = 'biobert_base_NER_NCBI-disease'
# MODEL = 'biobert/NCBI-disease'
# MODEL = 'BC5CDR-IOB'
# MODEL = 'biobert/ACD'
# MODEL = 'biobert/AnatEM-IOB'
# MODEL = 'biobert/NCBI-disease2'
# MODEL = 'biobert/BC5CDR-IOB'
# MODEL = 'biobert/AnatEM-BC5CDR'
MODEL = 'AnatEM-BC5CDR'



MODEL_PATH = '../../models/word_embeddings/'

MODEL_PATH += MODEL

In [3]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(MODEL_PATH)

labels = []
labels = [value for k, value in config.id2label.items()]

print(labels)

['B-Anatomy', 'B-Chemical', 'B-Disease', 'I-Anatomy', 'I-Chemical', 'I-Disease', 'O']


In [4]:
from transformers import BertForTokenClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, do_lower_case=False)

model = BertForTokenClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(labels),
    output_attentions = False,
    output_hidden_states = False
)

In [7]:
import torch
import numpy as np

# test_sentence = 'Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breathe and when I lay down on my bed. I suffer from arterial hypertension and smoke 20 cigarettes every day. My father had a “heart attack” at my age and I am very worried about it.'
# test_sentence = " Her eye is green. A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the eyes, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes. Alprazolan was administred."
test_sentence = 'Recently, we found that therapy with selegiline and L-dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline. '
# test_sentence = 'The cells were examined in a Zeiss LSM 510 laser scanning microscope equipped with a Plan - Apochromate 63x/1.4 oil immersion objective.'
# test_sentence = 'Torsade de pointes ventricular tachycardia during  low dose intermittent dobutamine treatment in a patient with dilated cardiomyopathy and congestive heart failure.'
# test_sentence = 'A DNA molecule 
# test_sentence = """"
# @NURSE
# > Doctor, we have a man (51 years old) who entered the emergency department reporting chest pain. His vital signs are ABP: 144x92mmHG; HR: 78bpm; RR: 21rpm; Temp: 37oC; O2Sat: 98%.
# @PATIENT
# > Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breathe and when I lay down on my bed. I suffer from arterial hypertension and smoke 20 cigarettes every day. My father had a "heart attack" at my age and I am very worried about it.
# <b>PHYSICAL EXAMINATION</b> <br> The cardiac and pulmonary auscultation are normal; chest pain does not worse with palpation of the thorax; there is no jugular stasis nor lower limb edema.
# Recently, we found that therapy with selegiline and L-dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline. 
# The cells were examined in a Zeiss LSM 510 laser scanning microscope equipped with a Plan - Apochromate 63x/1.4 oil immersion objective.
#  """


tokenized_sentence = tokenizer.encode(test_sentence)
# input_ids = torch.tensor([tokenized_sentence]).cuda()
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = model(input_ids)
# print(output)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(labels[label_idx])
        new_tokens.append(token)
for token, label in zip(new_tokens, new_labels):
    print("{}\t\t{}".format(label, token))


O		[CLS]
O		Recently
O		,
O		we
O		found
O		that
O		therapy
O		with
B-Chemical		selegiline
O		and
B-Chemical		L
I-Chemical		-
I-Chemical		dopa
O		was
O		associated
O		with
O		selective
B-Disease		systolic
I-Disease		orthostatic
I-Disease		hypotension
O		which
O		was
O		abolished
O		by
O		withdrawal
O		of
B-Chemical		selegiline
O		.
O		[SEP]
